# SD-TSIA 211 - TP 1 Report - HEYMANN Alexandre

## Least squares

### Copy data_center_help

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Loading data

data_matrix_train, COP_train, data_matrix_test, COP_test, names = np.load('data_center_data_matrix.npy', allow_pickle=True)

# Constructing matrices for min_w ||A w - b||_2**2

matrix_mean = np.mean(data_matrix_train, axis=0)
M = data_matrix_train - matrix_mean
matrix_std = np.std(M, axis=0)
M = M / matrix_std

A = np.hstack([M, np.ones((M.shape[0],1)), -(M.T * COP_train[:,3]).T])
b = COP_train[:,3]

# Constructing matrices for the test set

M_test = (data_matrix_test - matrix_mean) / matrix_std
A_test = np.hstack([M_test, np.ones((M_test.shape[0],1)), -(M_test.T * COP_test[:,3]).T])
b_test = COP_test[:,3]


# Loading raw data
import pandas as pd
data = pd.read_csv('Raw_Dataset_May.csv')

def name_to_subcategory_and_details(col_name):
    if np.isreal(col_name):
        col_name = names[col_name]
    indices = np.nonzero((data['NAME'] == col_name).values)[0]
    if len(indices) > 0:
        subcategory = data['SUBCATEGORY'].iloc[[indices[0]]].values[0]
        details = data['DETAILS'].iloc[[indices[0]]].values[0]
        return subcategory, details
    else:
        print('unknown name')



### <span style="color:red">Q1</span>



If $Aw=b$ then : 
\begin{align*}
    &(Aw)_t = b_t \\
\end{align*}

So :
\begin{align*}
    &y(t) = \tilde{x}(t)^Tw_1 + w_0 - y(t)\tilde{x}(t)^Tw_2 \quad \text{  for all } t \\
\end{align*}

And then : 

\begin{align*}
    &y(t) = \frac{w_1\tilde{x}(t)^T + w_0}{w_2\tilde{x}(t)^T + 1} \quad \text{  for all } t \\
\end{align*}

### <span style="color:red">Q2</span>

In [2]:
print(np.linalg.lstsq(A, b, rcond=None)[0])

[-0.00927821  0.08309371 -0.03672704 ...  0.01980595 -0.03057174
 -0.01188614]


### <span style="color:red">Q3</span>

In [3]:
# the predicted values
omega = np.linalg.lstsq(A, b, rcond=None)[0]
y_pred1 = np.dot(A_test, omega)

# Calculate mean squared error
mse_1 = np.mean((y_pred1 - b_test) ** 2)
print("Mean Squared Error :", mse_1)

Mean Squared Error : 780.8984793523316


### <span style="color:red">Q4</span>

In [4]:
from scipy.optimize import minimize
lambda_ = 100

def l2_regularization(w):
    return 1/2 * np.linalg.norm(A@w - b)**2 + lambda_/2 * np.linalg.norm(w)**2


w_initial = np.zeros(A.shape[1])

    # Minimize the objective function
result = minimize(l2_regularization, w_initial, method='L-BFGS-B')

w_opti = result.x
print('w_opti = ',w_opti)

y_pred2 = np.dot(A_test, w_opti)
mse_2 = np.mean((y_pred2 - b_test) ** 2)

mse_2

w_opti =  [-0.00160574  0.01476579  0.01046987 ...  0.00043573  0.00790493
 -0.00471621]


45.57535676269488

### <span style="color:red">Q5</span>

The function $f_1$ is defined as :
$$
f_1(w) = \frac{1}{2} \lVert Aw - b \rVert^2 + \frac{\lambda}{2} \lVert w \rVert^2
$$

So, the gradient of $f_1(w)$ is given by:
$$
\nabla f_1(w) = A^T(Aw - b) + \lambda w
$$

Assume $F_1$ :
$$
F_1(w) = \frac{1}{2} \lVert Aw - b \rVert^2
$$

Since $F_1$ is a quadratic function, its Hessian matrix is $A^TA$, which is always positive semi-definite. <u>Therefore, $F_1$ is convex</u>.

Assume $F_2$ :
$$
F_2(w) = \frac{\lambda}{2} \lVert w \rVert^2
$$

As a quadratic function, $F_2$ has a positive definite Hessian matrix $\lambda I$, where $I$ is the identity matrix. <u>Therefore,$F_2$ is convex</u>.

Now, the overall function $f_1$ is the sum of $F_1$ and $F_2$:
$$
f_1(w) = F_1(w) + F_2(w) = \frac{1}{2} \lVert Aw - b \rVert^2 + \frac{\lambda}{2} \lVert w \rVert^2
$$

Since both $F_1$ and $F_2$ are convex, their sum <u>$f_1$ is also convex</u>.


### <span style="color:red">Q6</span>

In [6]:
w = np.zeros(A.shape[1])
k = 0

L = np.linalg.norm(A.T @ A)
mu = lambda_
gamma = 2 / (L + mu)
print('gamma:', gamma, 'L:', L, 'mu:', mu)

alpha = gamma

while np.linalg.norm( A.T @ (A @ w - b) + lambda_ * w) > 1:
    gradient = A.T @ (A @ w - b) + lambda_ * w  # Calculate gradient
    w = w - alpha * gradient  # Update  vector
    k += 1

print("it took ", k, " iterations to converge")

gamma: 4.703467206086619e-07 L: 4252082.299499949 mu: 100
it took  59379  iterations to converge


## Regularization for a sparse model

### <span style="color:red">Q1</span>

The objective function $F_2$ can be written as the sum of two functions: $f_2$ and $g_2$, where $f_2$ is differentiable, and the proximal operator of $g_2$ is easy to compute. The proximal operator of a function $g$ is defined as:

$$
\text{prox}_g(v) = \arg \min_u \left( g(u) + \frac{1}{2}\|u - v\|_2^2 \right)
$$

For the Lasso problem, the objective function is:

$$
F_2(w) = \frac{1}{2} \|Aw - b\|_2^2 + \lambda \|w\|_1
$$

Let's write it in the required form $F_2 = f_2 + g_2$:

$$
F_2(w) = f_2(w) + g_2(w)
$$

* Differentiable part $f_2(w)$:

$$
f_2(w) = \frac{1}{2} \|Aw - b\|_2^2
$$

* Non-differentiable part $g_2(w)$:

$$
g_2(w) = \lambda \|w\|_1
$$

With the following

* Gradient of $f_2(w)$:

$$
\nabla f_2(w) = A^T(Aw - b)
$$

* Proximal operator of $g_2(w)$ = soft-thresholding operator $S_{\lambda}(v)$:

$$
\text{prox}_{g_2}(v)_i = \text{sign}(v_i) \max(|v_i| - \lambda, 0)
$$

### <span style="color:red">Q2</span>

In [8]:
def grad2(X):
    return A.T.dot(A.dot(X)-b)

def prox(x):
    return np.sign(x)*np.maximum(abs(x)-200*gamma,0)

X0=np.zeros(A.shape[1])

while np.linalg.norm(grad2(X0))>1:
    X0=prox(X0-gamma*grad2(X0))
    print(np.linalg.norm(grad2(X0)))

62374.8999505916
42067.79902278505
24103.91970635368
19600.643449466155
13430.648115481898
13189.89668080145
11466.572813428833
11581.543326569814
11107.075585558634
11056.80582201621
10867.373683030703
10764.264008566674
10645.857910435181
10536.630788349952
10431.555418471838
10334.171465390747
10238.537470383619
10151.357916749006
10066.133870441869
9981.195840369697
9899.494954078165
9819.781433000771
9745.522504608916
9670.450518038013
9599.837173169042
9534.951148766602
9471.051655121122
9409.804009541054
9348.676510596775
9290.040615509997
9234.684144207384
9179.549822191042
9126.634601728952
9074.81330757296
9020.889780135567
8969.737189764677
8919.910712263916
8870.428550261979
8822.505190230904
8777.201226662528
8733.49316442026
8689.568366720458
8647.155310104547
8605.65953556648
8565.44080934692
8521.02464292633
8481.800304799302
8445.020683323286
8407.209426835821
8370.210049652975
8334.514262848033
8297.557466756118
8261.398964011609
8226.767149816149
8192.136165583428
81

KeyboardInterrupt: 

## Choice of the regularization parameter

### <span style="color:red">Q1</span>